In [77]:
import spacy
import os
import sys
import skweak
import evaluation as evn
import itertools
from skweak.spacy import ModelAnnotator
from spacy.tokens import Doc, Span, Token
from typing import List, Set, Dict, Tuple, Optional
nlp = spacy.load("nl_core_news_md")

In [2]:
#TODAY I'LL TRAIN SOME SPACY MODELS - LET'S SEE... SONAR-1 AND CONLL-2002? 

#I ALSO HAVE TO MAKE A PRETRAINED CONLL-2002 XLM-R

In [203]:
countries = ['Afghanistan', 'Albanië', 'Algerije', 'Andorra', 'Angola', 'Antigua', 'Argentinië', 'Armenië', 'Australië', 'Oostenrijk',
             'Azerbeidzjan', "Bahama's", 'Bahrein', 'Bangladesh', 'Barbados', 'Wit-Rusland', 'België', 'Belize', 'Benin', 'Bhutan',
             'Bolivia', 'Bosnië-Herzegovina', 'Botswana', 'Brazilië', 'Brunei', 'Bulgarije', 'Burkina', 'Burundi', 'Cambodja', 'Kameroen',
             'Canada', 'Kaapverdië', 'Centraal-Afrikaanse Republiek', 'Tsjaad', 'Chili', 'China', 'Colombia', 'Comoren', 'Congo', 'Costa Rica',
             'Kroatië', 'Cuba', 'Cyprus', 'Tsjechië', 'Denemarken', 'Djibouti', 'Dominica', 'Dominicaanse Republiek', 'Oost-Timor',
             'Ecuador', 'Egypte', 'El Salvador', 'Equatoriaal-Guinea', 'Eritrea', 'Estland', 'Ethiopië', 'Fiji', 'Finland', 'Frankrijk',
             'Gabon', 'Gambia', 'Georgië', 'Duitsland', 'Ghana', 'Griekenland', 'Grenada', 'Guatemala', 'Guinee', 'Guinee-Bissau', 'Guyana',
             'Haïti', 'Honduras', 'Hongarije', 'IJsland', 'India', 'Indonesië', 'Iran', 'Irak', 'Ierland', 'Israël', 'Italië', 'Ivoorkust',
             'Jamaica', 'Japan', 'Jordanië', 'Kazachstan', 'Kenia', 'Kiribati', 'Noord-Korea', 'Zuid-Korea', 'Kosovo', 'Koeweit', 'Kirgizië',
             'Laos', 'Letland', 'Libanon', 'Lesotho', 'Liberia', 'Libië', 'Liechtenstein', 'Litouwen', 'Luxemburg', 'Macedonië', 'Madagaskar',
             'Malawi', 'Maleisië', 'Maldiven', 'Mali', 'Malta', 'Marshalleilanden', 'Mauritanië', 'Mauritius', 'Mexico', 'Micronesië',
             'Moldavië', 'Monaco', 'Mongolië', 'Montenegro', 'Marokko', 'Mozambique', 'Myanmar', 'Namibië', 'Nauru', 'Nepal', 'Nederland',
             'Nieuw-Zeeland', 'Nicaragua', 'Niger', 'Nigeria', 'Noorwegen', 'Oman', 'Pakistan', 'Palau', 'Panama', 'Papoea-Nieuw-Guinea',
             'Paraguay', 'Peru', 'Filipijnen', 'Polen', 'Portugal', 'Qatar', 'Roemenië', 'Russische Federatie', 'Rwanda', 'St Kitts & Nevis',
             'St Lucia', 'Saint Vincent en de Grenadines', 'Samoa', 'San Marino', 'Sao Tome & Principe', 'Saudi-Arabië', 'Senegal', 'Servië',
             'Seychellen', 'Sierra Leone', 'Singapore', 'Slowakije', 'Slovenië', 'Salomonseilanden', 'Somalië', 'Zuid-Afrika', 'Zuid-Soedan',
             'Spanje', 'Sri Lanka', 'Soedan', 'Suriname', 'Swaziland', 'Zweden', 'Zwitserland', 'Syrië', 'Taiwan', 'Tadzjikistan', 'Tanzania',
             'Thailand', 'Togo', 'Tonga', 'Trinidad en Tobago', 'Tunesië', 'Turkije', 'Turkmenistan', 'Tuvalu', 'Oeganda', 'Oekraïne',
             'Verenigde Arabische Emiraten', 'Verenigd Koninkrijk', 'Verenigde Staten', 'Uruguay', 'Oezbekistan', 'Vanuatu', 'Vaticaanstad', 'Venezuela',
             'Vietnam', 'Jemen', 'Zambia', 'Zimbabwe', 'VS', 'VK', 'Rusland', 'Zuid-Korea']

langs = ['Afar', 'Abchazisch', 'Avestan', 'Afrikaans', 'Akan', 'Amhaars', 'Aragonese', 'Arabisch', 'Aramees', 'Assamees', 'Avarisch', 'Aymara',
             'Azerbeidzjaans', 'Bashkir', 'Wit-Russisch', 'Bulgaars', 'Bambara', 'Bislama', 'Bengaals', 'Tibetaans', 'Bretons', 'Bosnisch', 'Kantonees',
             'Catalaans', 'Tsjetsjeens', 'Chamorro', 'Corsicaans', 'Cree', 'Tsjechisch', 'Tsjoevasj', 'Welsh', 'Deens', 'Duits', 'Divehi', 'Dzongkha', 'Ooi ',
             'Grieks', 'Engels', 'Esperanto', 'Spaans', 'Castiliaans', 'Ests', 'Baskisch', 'Perzisch', 'Fulah', 'Filipijns', 'Fins', 'Fijisch', 'Faeröers ',
             'Frans', 'West-Fries', 'Iers', 'Gaelic', 'Galicisch', 'Guarani', 'Gujarati', 'Manx', 'Hausa', 'Hebreeuws', 'Hindi', 'Hiri Motu',
             'Kroatisch', 'Haïtiaans', 'Hongaars', 'Armeens', 'Herero', 'Indonesisch', 'Igbo', 'Inupiaq', 'Ido', 'IJslands', 'Italiaans', 'Inuktitut',
             'Japans', 'Javaans', 'Georgisch', 'Kongo', 'Kikuyu', 'Kuanyama', 'Kazachs', 'Kalaallisut', 'Groenlands', 'Centraal Khmer', 'Kannada',
             'Koreaans', 'Kanuri', 'Kasjmiri', 'Koerdisch', 'Komi', 'Cornish', 'Kirgizisch', 'Latijn', 'Luxemburgs', 'Ganda', 'Limburgs', 'Lingala', 'Lao ',
             'Litouws', 'Luba-Katanga', 'Lets', 'Malagasi', 'Marshallese', 'Maori', 'Macedonisch', 'Malayalam', 'Mongools', 'Marathi', 'Maleis', 'Maltees', 'Birmaans', 'Nauru', 'Bokmål', 'Noors', 'Ndebele', 'Nepalees', 'Ndonga', 'Nederlands', 'Vlaams', 'Nynorsk', 'Navajo', 'Chichewa',
             'Occitaans', 'Ojibwa', 'Oromo', 'Oriya', 'Ossetisch', 'Punjabi', 'Pali', 'Pools', 'Pasjtoe', 'Portugees', 'Quechua', 'Reto-Romaans', 'Rundi ',
             'Roemeens', 'Russisch', 'Kinyarwanda', 'Sanskriet', 'Sardinisch', 'Sindhi', 'Sami', 'Sango', 'Singalees', 'Slowaaks', 'Sloveens', 'Samoaans',
             'Shona', 'Somalisch', 'Albanees', 'Servisch', 'Swati', 'Sotho', 'Soendanees', 'Zweeds', 'Swahili', 'Tamil', 'Telugu', 'Tadzjieks', 'Thais ',
             'Tigrinya', 'Turkmeens', 'Taiwanees', 'Tagalog', 'Tswana', 'Tonga', 'Turks', 'Tsonga', 'Tataars', 'Twi', 'Tahitiaans', 'Oeigoers', 'Oekraïens ',
             'Urdu', 'Oezbeeks', 'Venda', 'Vietnamees', 'Volapük', 'Wallonië', 'Wolof', 'Xhosa', 'Jiddisch', 'Yoruba', 'Zhuang', 'Mandarijn',
             'Mandarijn Chinees', 'Chinees', 'Zulu']

In [211]:
train = evn.to_spacy("data/ned_train.spacy")
val = evn.to_spacy("data/ned_testa.spacy")
test = evn.to_spacy("data/ned_testb.spacy")
sample = train[0]

In [206]:
def relabel(ent_label: str) -> str:
  """
  returns ConLL-2002 label of Spacy labelled entity
  """
  mappings = {"PERSON":"PER", "COMPANY":"ORG", "GPE":"LOC", 'EVENT':"MISC", 'FAC':"MISC", 'LANGUAGE':"MISC", 'LAW':"MISC", 'NORP':"MISC", 'PRODUCT':"MISC",'WORK_OF_ART':"MISC", "MISC":"MISC", "PER":"PER", "ORG":"ORG", "LOC":"LOC"}    
  exclude = {"CARDINAL", "ORDINAL", "DATE", "PERCENT", "QUANTITY", "TIME", "MONEY"}

  return mappings[ent_label] if ent_label != "" and ent_label not in exclude else "O"

class RelabelledModelAnnotator(ModelAnnotator):
  def __init__(self, name: str, model_path: str):
    super(RelabelledModelAnnotator, self).__init__(name, model_path)
    
  def find_spans(self, doc):
    # Create a new document (to avoid conflicting annotations)
    doc2 = self.create_new_doc(doc)
    # And run the model
    for _, proc in self.model.pipeline:
        doc2 = proc(doc2)
    # Add the annotation
    for ent in doc2.ents:
        #just put relabel function here
        yield ent.start, ent.end, relabel(ent.label_)
        
def country_annotator(doc: Doc):
  for tok in doc:
    if tok.text in countries:
      yield tok.i-1, tok.i+1, "LOC"
      
def nationality_annotator(doc: Doc):
  for tok in doc:
    if tok.text in countries:
      yield tok.i-1, tok.i+1, "MISC"
      
country_annotate = skweak.heuristics.FunctionAnnotator("countries", country_annotator)
nationality_annotate = skweak.heuristics.FunctionAnnotator("nationality", nationality_annotator)

In [6]:
#model annotators
spacy_nl = RelabelledModelAnnotator("spacy", "nl_core_news_md")
spacy_conll = RelabelledModelAnnotator("conll", "models/conll2002_spacy")

In [7]:
#gazetteers 
geonames = skweak.gazetteers.extract_json_data("data/geonames.json")
geonames_annotator = skweak.gazetteers.GazetteerAnnotator("geonames", geonames)

nederlocs = skweak.gazetteers.extract_json_data("data/nl.json")
dutch_loc_annotator = skweak.gazetteers.GazetteerAnnotator("nederlocs", nederlocs)

crunchbase = skweak.gazetteers.extract_json_data("data/crunchbase_alt.json")
crunchbase_annotator = skweak.gazetteers.GazetteerAnnotator("crunchbase", crunchbase)

Extracting data from data/geonames.json
Populating trie for class LOC (number: 15205)
Extracting data from data/nl.json
Populating trie for class LOC (number: 22819)
Extracting data from data/crunchbase_alt.json
Populating trie for class PER (number: 1062669)
Populating trie for class ORG (number: 789205)


In [8]:
#functions 
import json
import spacy_wrapper

NAME_PREFIXES = {"-", "von", "van", "de", "di", "le", "la", "het", "'t'", "dem", "der", "den", "d'", "ter"}

class SpanGenerator:
    """Generate spans that satisfy a token-level constratint. From Lison et al. 2020"""
    
    def __init__(self, constraint, label="ENT", exceptions=("'s", "-")):
        """annotation with a constraint (on spacy tokens). Exceptions are sets of tokens that are allowed
        to violate the constraint inside the span"""
        
        self.constraint = constraint
        self.label = label
        self.exceptions = set(exceptions)
        
    def __call__(self, spacy_doc):    

        i = 0
        while i < len(spacy_doc):
            tok = spacy_doc[i]
                # We search for the longest span that satisfy the constraint
            if self.constraint(tok):
                j = i+1
                while True:
                    if j < len(spacy_doc) and self.constraint(spacy_doc[j]):
                        j += 1
                    # We relax the constraint a bit to allow genitive and dashes
                    elif j < (len(spacy_doc)-1) and spacy_doc[j].text in self.exceptions and self.constraint(spacy_doc[j+1]):
                        j += 2
                    else:
                        break

                # To avoid too many FPs, we only keep entities with at least 3 characters (excluding punctuation)
                if len(spacy_doc[i:j].text.rstrip(".")) > 2:
                    yield i, j, self.label
                i = j
            else:
                i += 1

class FullNameGenerator:
    """Search for occurrences of full person names (first name followed by at least one title token). From Lison et al. 2020"""

    def __init__(self):
        fd = open("data/first_names.json")
        self.first_names = set(json.load(fd))
        fd.close()
        self.suggest_generator = SpanGenerator(lambda x: is_likely_proper(x), 
                                               exceptions=NAME_PREFIXES)
        
    def __call__(self, spacy_doc):
        for start, end, _ in self.suggest_generator(spacy_doc):  
            # We assume full names are between 2 and 4 tokens
            if (end-start) < 2 or (end-start) > 5:
                continue
                
            elif (spacy_doc[start].text in self.first_names and spacy_doc[end-1].is_alpha 
                  and spacy_doc[end-1].is_title): 
                yield start, end, "PER"

def in_compound(tok):
    """Returns true if the spacy token is part of a compound phrase"""
    if tok.dep_=="compound":
        return True
    elif tok.i > 0 and tok.nbor(-1).dep_=="compound":
        return True
    return False
  
def is_likely_proper(tok):
    """Returns true if the spacy token is a likely proper name, based on its form."""
    if len(tok)< 2:
        return False
    
    # If the lemma is titled, just return True
    elif tok.lemma_.istitle():
        return True
       
    # Handling cases such as iPad
    elif len(tok)>2 and tok.text[0].islower() and tok.text[1].isupper() and tok.text[2:].islower():
        return True
    
    elif (tok.is_upper and tok.text not in spacy_wrapper.CURRENCY_CODES 
          and tok.text not in spacy_wrapper.NOT_NAMED_ENTITIES):
        return True
    
    # Else, check whether the surface token is titled and is not sentence-initial
    elif (tok.i > 0 and tok.is_title and not tok.is_sent_start and tok.nbor(-1).text not in {'\'', '"', '‘', '“', '”', '’'} 
          and not tok.nbor(-1).text.endswith(".")):
        return True
    return False  

fullname = FullNameGenerator()
fullname_annotator = skweak.heuristics.FunctionAnnotator("fullname_detector", fullname)

In [210]:
def process_doc(doc: Doc) -> Doc:
  #apply all labelling functions to doc
  #return nationality_annotate(country_annotate(spacy_conll(dutch_loc_annotator(geonames_annotator(crunchbase_annotator(fullname_annotator(spacy_nl(doc))))))))
  return spacy_nl(doc)
  #return #dutch_loc_annotator(crunchbase_annotator(fullname_annotator(doc)))

def process_docs(docs: List[Doc]) -> Doc:
  docs = [process_doc(doc) for doc in docs]
  hmm = skweak.aggregation.HMM("hmm", ["PER", "ORG", "LOC", "MISC"])
  hmm.fit_and_aggregate(docs)
  for doc in docs:
    doc.ents = doc.spans['hmm']
  return docs

In [153]:
sample = nlp("Hoi ik ben William, 23 jaar en ik woon in Amsterdam maar ben nu in Heerenveen. Ik ga naar de Universiteit van Amsterdam. Nederland, Amerika, Belgie.")

processed = process_docs([sample])
  
skweak.utils.display_entities(processed[0])


Starting iteration 1
Finished E-step with 1 documents
Starting iteration 2
Finished E-step with 1 documents


         1         -31.4380             +nan
         2         -31.4380          +0.0000


In [212]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score

mapper = {"O" : 0,
          "PER" : 1, 
          "LOC" : 2, 
          "ORG" : 3,
          "MISC" : 4}

def ent_iob(tok: Token) -> str:
  """
  returns ent IOB label
  """
  ent_type = "-" + tok.ent_type_ if tok.ent_type else ""
  return tok.ent_iob_ + ent_type

def ent_type(tok: Token) -> str:
  """
  returns entity type for given token
  """
  return tok.ent_type_ if tok.ent_type_ else tok.ent_iob_

def doc_ents(doc: Doc) -> List[str]:
  """
  returns list of IOB style formatted entities in doc
  """
  return [ent_iob(tok) for tok in doc]
  
def score_annotator(docs: List[Doc]) -> dict:
  """
  given docs, applies annotator to docs and evaluates against ground truth 
  """
  true = [token for doc in docs for token in doc_ents(doc)]
  
  annotated = process_docs(docs)
  pred = [token for doc in annotated for token in doc_ents(doc)]
  
  #calculate precision, recall, f1 and give it weights to micro average
  #OR add all predictions to one humongous list and run functions on that list 

  labels = ["B-PER", "I-PER", "B-LOC", "I-LOC", "B-ORG", "I-ORG", "B-MISC", "I-MISC"]
  return {"precision" : precision_score(true, pred, labels=labels, average="micro"), 
          "recall" : recall_score(true, pred, labels=labels, average="micro"),
          "f1" : f1_score(true, pred, labels=labels, average="micro")}


sample = test
score_annotator(sample)

Starting iteration 1
Finished E-step with 119 documents
Starting iteration 2


         1      -21240.6415             +nan


Finished E-step with 119 documents


         2      -21240.6415          -0.0000


{'precision': 0.6441116657422814,
 'recall': 0.605071205279611,
 'f1': 0.6239813736903377}

In [122]:
a = np.array(["a", "b", "c", "d", "e"], dtype="object")
b = np.array(["a", "a", "c", "c", "c"], dtype="object")
cm = confusion_matrix(a, b)

In [127]:


cm

0.4000000000000001

In [ ]:

from datasets import Dataset, ClassLabel, Sequence
doc = nlp(sample.text)

def name_detector_fn(doc):
    names = ["David", "Cameron", "William", "Bos"]
    for token in doc: 
        if token.text in names:
            yield token.i, token.i+1, "PER"

name_detector = skweak.heuristics.FunctionAnnotator("name_detector", name_detector_fn)

geonames = skweak.gazetteers.extract_json_data("data/geonames.json", spacy_model="en_core_web_sm")
geonames_annotator = skweak.gazetteers.GazetteerAnnotator("locations", geonames)


#hmm = skweak.aggregation.HMM("hmm", ["PER", "ORG", "LOC", "MISC"])
#hmm.fit_and_aggregate([doc])
#spans = doc.spans["hmm"]
#doc.ents = spans

#for span in doc.ents:
#  print(span, span.label, span.start, span.end, span.doc[span.start : span.end])

In [ ]:
type(geonames_annotator)

In [ ]:
from datasets import Dataset, ClassLabel, Sequence

def relabel(ent_label: str) -> str:
  """
  returns ConLL-2002 label of Spacy labelled entity
  """
  mappings = {"PERSON":"PER", "COMPANY":"ORG", "GPE":"LOC", 'EVENT':"MISC", 'FAC':"MISC", 'LANGUAGE':"MISC", 'LAW':"MISC", 'NORP':"MISC", 'PRODUCT':"MISC",'WORK_OF_ART':"MISC", "MISC":"MISC", "PER":"PER", "ORG":"ORG", "LOC":"LOC"}    
  exclude = {"CARDINAL", "ORDINAL", "DATE", "PERCENT", "QUANTITY", "TIME", "MONEY"}

  return mappings[ent_label] if ent_label != "" and ent_label not in exclude else None

def convert_ent(token) -> str:
  """
  returns ConLL-2002 IOB style entity label of Spacy token
  """
  return token.ent_iob_ + "-" + relabel(token.ent_type_) if relabel(token.ent_type_) else token.ent_iob_

def process_spacy(docs: list):
  store = []
  tokens = []
  ids = []

  c = 0
  classlabels = ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'])
  for doc in docs:
    ents = [classlabels.str2int(convert_ent(tok)) for tok in doc]
    toks = [token.text for token in doc]
    store.append(ents)
    tokens.append(toks)
    ids.append(str(c))
    c += 1 
    
  d = {"ids" : ids,
       "ner_tags" : store,
       "tokens" : tokens}

  class_sequence = Sequence(feature =  classlabels, id = None)
  ds = Dataset.from_dict(d)
  ds.features["ner_tags"] = class_sequence
  return ds

ds = process_spacy([doc])
ds

In [ ]:
ds[0]

In [ ]:
import tarfile

# We retrieve the texts from Wablieft tarfile
texts = [] 
archive_file = tarfile.open("data/plain.tar.xz")
for archive_member in archive_file.getnames():
    if archive_member.endswith(".txt"):
        #weird encoding because of Turkish letters
        text = archive_file.extractfile(archive_member).read().decode("cp850")
        texts.append(text)

In [ ]:
docs = list(nlp.pipe(texts))

In [95]:
def name_detector_fn(doc):
    names = ["David", "Cameron", "William", "Bos"]
    for token in doc: 
        if token.text in names:
            yield token.i, token.i+1, "PER"

name_detector = skweak.heuristics.FunctionAnnotator("name_detector", name_detector_fn)

def address_detector_fn(doc):
    for token in doc: 
        if token.text[0].isupper() and doc[token.i+1].is_digit:
            yield token.i, token.i+2, "LOCATION"

address_detector = skweak.heuristics.FunctionAnnotator("address_detector", address_detector_fn)

def company_detector_fn(doc):
    companies = ["Microsoft", "Apple", "Gemeente Amsterdam", "Universiteit van Amsterdam", "UvA"]
    for token in doc:
        if token.text in companies:
            yield token.i, token.i+1, "ORG"

company_detector = skweak.heuristics.FunctionAnnotator("company_detector", company_detector_fn)

names = skweak.gazetteers.extract_json_data("data/geonames.json", spacy_model="en_core_web_sm")
name_annotator = skweak.gazetteers.GazetteerAnnotator("locations", names)

Extracting data from data/geonames.json
Populating trie for class GPE (number: 15205)


In [98]:
type(address_detector) == skweak.heuristics.FunctionAnnotator

True

In [ ]:
processed = list(name_detector.pipe(name_annotator.pipe(docs[:25])))
hmm = skweak.aggregation.HMM("hmm", ["PERSON", "LOCATION", "ORG", "GPE"])

#skweak.utils.display_entities(processed[7], "locations")
hmm.fit_and_aggregate(processed)
skweak.utils.display_entities(processed[7], "hmm", add_tooltip=True)

In [ ]:
for doc in processed:
    if "hmm" in doc.spans.keys():
        doc.ents = doc.spans["hmm"]
    else: 
        doc.ents = ()

docs = docs[:100]
skweak.utils.docbin_writer(docs, "data/wablieft.spacy")

In [ ]:
!spacy init fill-config base_config.cfg config.cfg

In [ ]:
!spacy train config.cfg --paths.train data/wablieft.spacy --paths.dev data/wablieft.spacy --initialize.vectors nl_core_news_md --output output

In [ ]:
model = spacy.load("output/model-best")

In [ ]:
doc = processed[7]

In [ ]:
skweak.utils.display_entities(model("Hoi ik ben William en ik woon in New York. Ik kom uit Flanders en ben geboren in Antwerpen."))

In [ ]:
from lison2020 import annotations
from spacy.tokens import DocBin
test = DocBin().from_disk("data/ned_testb.spacy")
sys.path.insert(0, './lison2020')

In [ ]:
annotator = annotations.FullAnnotator().add_all()

In [ ]:
import evaluation

In [ ]:
evaluation.spacy_benchmark("data/ned_testb.spacy")